<a href="https://colab.research.google.com/github/rupashibahl/LifePlanner/blob/master/lifeplannergui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [106]:
import ipywidgets as widgets
from ipywidgets import *
from IPython.display import display, clear_output

In [116]:
from sqlalchemy import create_engine, Column, Integer, String, Date, Time, ForeignKey
from sqlalchemy.orm import sessionmaker, declarative_base, relationship
from datetime import time as dtime, datetime

In [108]:
Base = declarative_base()
engine = create_engine('sqlite:///lifeplannerdatabase.db')

In [109]:
class User(Base):
    __tablename__ = 'users'
    id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String, nullable=False, unique =True)
    tasks = relationship('Task', back_populates='user', cascade='all, delete')
    def __repr__(self):
        return f'<User(id={self.id}, name={self.name})>'

In [110]:
class Task(Base):
    __tablename__ = 'tasks'
    id = Column(Integer, primary_key=True, autoincrement=True)
    user_id = Column(Integer, ForeignKey('users.id'), nullable=False)
    user = relationship('User', back_populates='tasks')
    date = Column(Date, nullable=False)
    time = Column(Time)
    title = Column(String, nullable=False)
    description = Column(String)
    def __repr__(self):
        return f'<Task(id={self.id}, user_id={self.user_id}, date={self.date}, title={self.title})>'

In [111]:
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
session = Session()

In [112]:
current_user = None
current_date = None

In [118]:
def show_message(msg: str):
    with output_area:
        clear_output(wait=True)
        display(widgets.HTML(f"<b>{msg}</b>"))

card_layout = Layout(
    width="650px",
    border="1px solid #444",
    border_radius="10px",
    padding="12px 16px",
    margin="8px auto",
)
button_layout = Layout(width="180px", height="40px")
small_button_layout = Layout(width="260px", height="36px")
tasks_layout = Layout(width="100%", height="200px")

output_area = widgets.Output(layout=Layout(width="650px", margin="8px auto", border="2px solid lightblue", padding="10px"))

user_input = widgets.Text(placeholder="Full Name", description="")
date = widgets.DatePicker(description="")

load_button = widgets.Button(
    description="Enter", button_style="success", layout=button_layout
)

tasks = widgets.SelectMultiple(
    description="", options=[], layout=tasks_layout
)

add_button = widgets.Button(
    description="Add Task", button_style="success", layout=button_layout
)
edt_button = widgets.Button(
    description="Edit Task", button_style="", layout=button_layout
)
delete_button = widgets.Button(
    description="Remove Task", button_style="warning", layout=button_layout
)

confirm_delete_button = widgets.Button(
    description="Confirm Delete Selected Task(s)",
    button_style="danger",
    layout=small_button_layout,
    disabled=True,
)
confirm_delete_button.layout.display = "none"

header = widgets.HTML(
    "<h2 style='text-align:center; margin-top:10px; margin-bottom:4px;'>Life Planner</h2>"
)


user_date_card = VBox(
    [
        widgets.HTML("<b>User & Date</b>"),
        HBox(
            [widgets.Label("User:", layout=Layout(width="80px")), user_input],
            layout=Layout(width="100%", justify_content="flex-start"),
        ),
        HBox(
            [
                widgets.Label("Date:", layout=Layout(width="80px")),
                date,
                load_button,
            ],
            layout=Layout(
                width="100%", justify_content="flex-start", align_items="center"
            ),
        ),
    ],
    layout=card_layout,
)

tasks_card = VBox(
    [
        widgets.HTML("<b>Your Tasks</b>"),
        tasks,
        HBox(
            [add_button, edt_button, delete_button],
            layout=Layout(
                width="100%", justify_content="space-between", margin="6px 0"
            ),
        ),
        confirm_delete_button,
    ],
    layout=card_layout,
)

pre_ui = VBox(
    [header, user_date_card, output_area],
    layout=Layout(align_items="center"),
)
main_ui = VBox(
    [header, user_date_card, tasks_card, output_area],
    layout=Layout(align_items="center"),
)

container = VBox([pre_ui])


def refresh_tasks():
    global current_user, current_date
    if current_user is None or current_date is None:
        tasks.options = []
        tasks.value = ()
        return

    daily_tasks = (
        session.query(Task)
        .filter_by(user_id=current_user.id, date=current_date)
        .order_by(Task.time.is_(None), Task.time)
        .all()
    )

    options = []
    for t in daily_tasks:
        if t.time:
            label = f"{t.title} ({t.time.strftime('%H:%M')})"
        else:
            label = t.title
        options.append((label, t.id))

    tasks.options = options
    tasks.value = ()

In [119]:
def on_button_clicked(b):
    global current_user, current_date

    user_name = user_input.value.strip()
    chosen_date = date.value

    if not user_name or not chosen_date:
        show_message(
            "Please enter your full name and select the date before you click Enter."
        )
        return

    user = session.query(User).filter_by(name=user_name).first()
    if user is None:
        user = User(name=user_name)
        session.add(user)
        session.commit()

    current_user = user
    current_date = chosen_date

    refresh_tasks()
    container.children = [main_ui]
    show_message(f"Loaded tasks for {current_user.name} on {current_date.isoformat()}.")


def on_add_clicked(b):
    global current_user, current_date
    if current_user is None or current_date is None:
        show_message("Please enter your name and pick a date, then click Enter first.")
        return

    show_message("Opening 'Add New Task' form...")

    title_input = widgets.Text(description="", placeholder="Title")
    desc_input = widgets.Textarea(description="", placeholder="Description")
    time_input = widgets.Text(description="", placeholder="HH:MM (optional)", value="") # Changed to Text
    save_btn = widgets.Button(description="Save", button_style="success")
    cancel_btn = widgets.Button(description="Cancel")

    form = VBox(
        [
            widgets.HTML("<b>Add New Task</b>"),
            widgets.Label("Title"),
            title_input,
            widgets.Label("Description"),
            desc_input,
            widgets.Label("Time (optional)"),
            time_input,
            HBox([save_btn, cancel_btn], layout=Layout(gap="8px")),
        ]
    )

    def save_new_task(btn):
        title_val = title_input.value.strip()
        desc_val = desc_input.value.strip()
        time_val_str = time_input.value.strip()
        time_val = None

        if not title_val:
            show_message("Task title cannot be empty.")
            return

        if time_val_str:
            try:
                time_val = datetime.strptime(time_val_str, '%H:%M').time()
            except ValueError:
                show_message("Invalid time format. Please use HH:MM (e.g., 14:30) or leave blank.")
                return

        new_task = Task(
            user_id=current_user.id,
            date=current_date,
            time=time_val,
            title=title_val,
            description=desc_val,
        )
        session.add(new_task)
        session.commit()

        refresh_tasks()
        show_message(f"Task '{title_val}' added.")

    def cancel_add(btn):
        show_message("Cancelled adding task.")

    save_btn.on_click(save_new_task)
    cancel_btn.on_click(cancel_add)

    with output_area:
        clear_output()
        display(form)


def on_edit_clicked(b):
    global current_user, current_date
    if current_user is None or current_date is None:
        show_message("Please enter your name and pick a date, then click Enter first.")
        return

    if len(tasks.value) == 0:
        show_message("Please select a task to edit.")
        return
    if len(tasks.value) > 1:
        show_message("Please select exactly ONE task to edit.")
        return

    task_id = list(tasks.value)[0]
    task_obj = session.query(Task).get(task_id)
    if task_obj is None:
        show_message("Could not find that task in the database.")
        return

    title_input = widgets.Text(description="", value=task_obj.title)
    desc_input = widgets.Textarea(description="", value=task_obj.description or "")
    # Changed to Text, format existing time to HH:MM if it exists
    time_input = widgets.Text(description="", placeholder="HH:MM (optional)", value=task_obj.time.strftime('%H:%M') if task_obj.time else "")
    save_btn = widgets.Button(description="Update", button_style="success")
    cancel_btn = widgets.Button(description="Cancel")

    form = VBox(
        [
            widgets.HTML("<b>Edit Task</b>"),
            widgets.Label("Title"),
            title_input,
            widgets.Label("Description"),
            desc_input,
            widgets.Label("Time (optional)"),
            time_input,
            HBox([save_btn, cancel_btn], layout=Layout(gap="8px")),
        ]
    )

    def save_edit(btn):
        title_val = title_input.value.strip()
        desc_val = desc_input.value.strip()
        time_val_str = time_input.value.strip()
        time_val = None

        if not title_val:
            show_message("Task title cannot be empty.")
            return

        if time_val_str:
            try:
                time_val = datetime.strptime(time_val_str, '%H:%M').time()
            except ValueError:
                show_message("Invalid time format. Please use HH:MM (e.g., 14:30) or leave blank.")
                return

        task_obj.title = title_val
        task_obj.description = desc_val
        task_obj.time = time_val

        session.commit()
        refresh_tasks()
        show_message(f"Task '{title_val}' updated.")

    def cancel_edit(btn):
        show_message("Cancelled editing task.")

    save_btn.on_click(save_edit)
    cancel_btn.on_click(cancel_edit)

    with output_area:
        clear_output()
        display(form)


def on_delete_clicked(b):
    if len(tasks.value) == 0:
        show_message("Please select at least one task to delete.")
        confirm_delete_button.disabled = True
        confirm_delete_button.layout.display = "none"
        return

    confirm_delete_button.disabled = False
    confirm_delete_button.layout.display = ""  # show it
    show_message(
        "Press 'Confirm Delete Selected Task(s)' to permanently delete the selected tasks."
    )


def on_confirm_delete_clicked(b):
    if len(tasks.value) == 0:
        show_message("No tasks selected.")
        confirm_delete_button.disabled = True
        confirm_delete_button.layout.display = "none"
        return

    task_ids = list(tasks.value)
    for tid in task_ids:
        task_obj = session.query(Task).get(tid)
        if task_obj:
            session.delete(task_obj)
    session.commit()

    refresh_tasks()
    confirm_delete_button.disabled = True
    confirm_delete_button.layout.display = "none"
    show_message("Selected task(s) deleted.")


load_button.on_click(on_button_clicked)
add_button.on_click(on_add_clicked)
edt_button.on_click(on_edit_clicked)
delete_button.on_click(on_delete_clicked)
confirm_delete_button.on_click(on_confirm_delete_clicked)

In [120]:
display(container)

/tmp/ipython-input-3980166175.py:109: LegacyAPIWarning: The Query.get() method is considered legacy as of the 1.x series of SQLAlchemy and becomes a legacy construct in 2.0. The method is now available as Session.get() (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  task_obj = session.query(Task).get(task_id)
/tmp/ipython-input-3980166175.py:193: LegacyAPIWarning: The Query.get() method is considered legacy as of the 1.x series of SQLAlchemy and becomes a legacy construct in 2.0. The method is now available as Session.get() (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  task_obj = session.query(Task).get(tid)
